# **LAB 3. TEXT CLASSIFICATION** 

### **<font color=green>INSTRUCTIONS:</font>** <br> 

**<font color=green> 1. Look for EXERCISES in the script.</font>** <br>

**<font color=green> 2. Each student INDIVIDUALLY uploads this script with their answers embedded to Canvas by the end of the lab session (recommended) or by Wednesday, 11:59pm Central Time.</font>** 

<font color=green>**Note:** Jupyter cloud can be found here if you need it: [jupyter.org/try](jupyter.org/try). You are likely to need it if you attend remotely from China (to access the web-based dataset we use in the lab). </font>

### Lab Objective
Our objective is to classify consumer messages based on the topic of the message. Today, you will:<br>
1. Learn how to vectorize *training* data and *testing* data (**watch out for very important nuances in the script!**)
2. Do feature selection using **filter** methods: chi-squared statistics and a variation on the entropy statistic called mutual information.
3. Train and test a **Naive Bayes classifier** for text data (you can use other methods, such as Support Vector Machines).


### Session Prep
Install the modules we'll need:

In [2]:
import sys

#instruct to not show the deprication warnings about future changes in the modules
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

!{sys.executable} -m pip install numpy
import numpy as np

!{sys.executable} -m pip install sklearn
from sklearn import metrics

!{sys.executable} -m pip install pandas
import pandas as pd

!{sys.executable} -m pip install nltk
import nltk

Define the text normalization function from Text_Normalization_Function.ipynb we used in Lab 2 (Important: Make sure Text_Normalization_Function.ipynb file is in the same directory as the current notebook):

In [3]:
%run ./Text_Normalization_Function.ipynb #defining text normalization function

[nltk_data] Downloading package stopwords to /Users/lilia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lilia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lilia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/lilia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  ['<', 'p', '>', 'The', 'circus', 'dog', 'in', 'a', 'plissé', 'skirt', 'jumped', 'over', 'Python', 'who', 'was', "n't", 'that', 'large', ',', 'just', '3', 'feet', 'long.', '<', '/p', '>']
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  <p>The circus dog in a plissé skirt jumped over Python who was not that large, just 3 feet long.</p>
Original:   <p>The circus dog in a plissé skirt jumped over Python who wasn't that large, just 3 feet long.</p>
Processed:  [('<', 'a'), ('p', 'n'), ('>', 'v'), ('the', None), ('circus', 'n'), ('dog', 'n'), ('in', None), ('a', None), ('plissé', 'n'), ('skirt', 'n'), ('jumped', 'v'), ('over', None), ('python', 'n'), ('who', None), ('was', 'v'), ("n't", 'r'), ('that', None), ('large', 'a'), (',', None), ('just', 'r'), ('3', None), ('feet', 'n'), ('long.', 'a'), 

### Downloading and Exploring Data

**Important if you are attending remotely from China: To access the data from a website below, you might need to use VPN (might be slow) or Jupyter cloud at [jupyter.org/try](jupyter.org/try).**<br>

Download the dataset from the dataset sklearn's collection of datasets (sklearn.datasets). The dataset we need is called fetch_20newsgroups:<br>

You can check out the documentation for the dataset here: https://bit.ly/3aM5tUo

In [4]:
from sklearn.datasets import fetch_20newsgroups

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Out of 20 newsgroups (topics) available, we will use posts on **4** topics: <br><br>-atheism<br> -religion<br> -computer graphics<br>-science<br><br> You can refer to those newsgroups as **classes** or **categories**. Note right away that the "atheism" and "religion" classes are likely **similar** to each other: people might be using similar words when they talk about atheism and religion.

In [5]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Now, let's create our **training** and **testing** datasets. We do that by picking the posts belonging to the selected classes, marked in the dataset "test" or "train". We remove headers (likely, email or letter headers), footers (containing author's signatures, etc.), and quotes: 

In [6]:
corpus_train = fetch_20newsgroups(categories = categories,
                                  subset = 'train', 
                                  remove = ('headers', 'footers', 'quotes')) 

corpus_test = fetch_20newsgroups(categories = categories,
                                 subset='test', 
                                 remove=('headers', 'footers', 'quotes')) 

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's inspect the data by looking at the training text corpus. First, have a look at one of the posts:

In [7]:
print(corpus_train.data[7])       


Acorn Replay running on a 25MHz ARM 3 processor (the ARM 3 is about 20% slower
than the ARM 6) does this in software (off a standard CD-ROM). 16 bit colour at
about the same resolution (so what if the computer only has 8 bit colour
support, real-time dithering too...). The 3D0/O is supposed to have a couple of
DSPs - the ARM being used for housekeeping.


A 25MHz ARM 6xx should clock around 20 ARM MIPS, say 18 flat out. Depends
really on the surrounding system and whether you are talking ARM6x or ARM6xx
(the latter has a cache, and so is essential to run at this kind of speed with
slower memory).

I'll stop saying things there 'cos I'll hopefully be working for ARM after
graduation...

Mike

PS Don't pay heed to what reps from Philips say; if the 3D0/O doesn't beat the
   pants off 3DI then I'll eat this postscript.


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The class labels for each message that you will be using for training and testing are encoded as numbers and can be accessed via attribute **.target** and their names can be accessed via attribute **.target_names**:

In [8]:
print("Category names: ", corpus_train.target_names)    
print("Categories for first 10 observations: ", corpus_train.target[:10])     
print("Number of posts in the training dataset: ", corpus_train.filenames.shape[0]) 

Category names:  ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
Categories for first 10 observations:  [1 3 2 0 2 0 2 1 2 1]
Number of posts in the training dataset:  2034


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's now define a function and call it **fmat_descr_fun** to be used later to describe the feature matrix (vectorized corpus). The function prints out the dimensions of the matrix, share of non-zero elements in the matrix and so on. The function takes two inputs: the feature matrix and your vectorizer function. <br><br>In what follows later, you will see the descriptives that the function produces:

In [9]:
def fmat_descr_fun(your_feature_matrix,your_vectorizer):
    print("Dimensions (number of posts, number of features): ", your_feature_matrix.shape)  
    print("The first 5 features - names: ", your_vectorizer.get_feature_names()[0:5]) 
    print("Share of non-zero elements in the matrix: ", 
          your_feature_matrix.nnz / (float(your_feature_matrix.shape[0]) * float(your_feature_matrix.shape[1])))
    print("Average number of features present, per post: ", 
          round(your_feature_matrix.nnz/float(your_feature_matrix.shape[0]),1))

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Feature Extraction for TRAINING Data

Let's do feature extraction for our **TRAINING** data using the **"Bag-of-words"** method and **TF-IDF** method (optional).

#### "Bag-of-words" Vectorization for TRAINING data

As you remember from Lab 2, to do the Bag-of-Words vectorization, we can use the CountVectorizer function from the sklearn package: 

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's define our Bag-of-Words vectorizer:

In [11]:
bow_vectorizer = CountVectorizer()

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Now we can apply the vectorizer we defined, bow_vectorizer, to our training dataset, **without normalizing** it first (though tokenization will be done by the vectorizer). Remember, to create the corpus vocabulary and to vectorize the data accroding to that vocabulary, we use the **.fit_transform** method:

In [12]:
corpus_train_bow = bow_vectorizer.fit_transform(corpus_train.data)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Describe the Bag-of-Words matrix that you got by applying the function **fmat_descr_fun** we defined above:

In [13]:
fmat_descr_fun(corpus_train_bow, bow_vectorizer)

Dimensions (number of posts, number of features):  (2034, 26879)
The first 5 features - names:  ['00', '000', '0000', '00000', '000000']
Share of non-zero elements in the matrix:  0.0035978272269590263
Average number of features present, per post:  96.7


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's have a look at the first 5 rows in the resulting matrix:

In [14]:
#convert vectorized data to a dataframe and give columns their names
corpus_train_bow_table = pd.DataFrame(data = corpus_train_bow.todense(), columns = bow_vectorizer.get_feature_names())
corpus_train_bow_table.head(5)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,00,000,0000,00000,000000,000005102000,000062david42,0001,000100255pixel,00041032,...,zurich,zurvanism,zus,zvi,zwaartepunten,zwak,zwakke,zware,zwarte,zyxel
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **<font color=green>EXERCISE 1:</font>**
**<font color=green> Answer the following questions using the descriptives of the Bag-of-Words matrix:</font>** <br><br>
**<font color=green>1.1. How many features does the Bag-of-Words matrix contain?</font>** <br><br> 
Your answer: 26879

26879

**<font color=green>1.2. Is the Bag-of-Words matrix sparse? Explain your answer: </font>** <br><br> 
Your answer: Yes. Many of the words have frequency 0.

Yes. Many of the words have frequency 0.

Let's normalize our training data and call the normalized corpus **NORM_corpus_train**. Note that it will take some time for the normalization function to finish its job as you are working with large dataset.

In [15]:
NORM_corpus_train = normalize_corpus(corpus_train.data)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### **<font color=green>EXERCISE 2:</font>**
**<font color=green>First, vectorize the normalized training corpus (NORM_corpus_train) using the Bag-of-Words approach and name the vectorized corpus NORM_corpus_train_bow.** <br><br>
Your code:    

In [16]:
NORM_corpus_train_bow = bow_vectorizer.fit_transform(NORM_corpus_train)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**<font color=green>
Second, describe the normalized training corpus using the fmat_descr_fun function.** <br><br> Your code:

In [17]:
fmat_descr_fun(NORM_corpus_train_bow, bow_vectorizer)

Dimensions (number of posts, number of features):  (2034, 21061)
The first 5 features - names:  ['000062david42', '000100255pixel', '000usd', '001200201pixel', '00index']
Share of non-zero elements in the matrix:  0.002958676433492318
Average number of features present, per post:  62.3


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**<font color=green>2.1. What differences do you see between normalized and non-normalized feature matrices?</font>** <br><br>
Your answer:

The number of features decreses to 21061 from 26897. And the share of non-zero elements drops by anout 0.1%

### Feature Selection for TRAINING Data
Let's select the best features for your normalized training corpus, **NORM_corpus_train_bow**, using the **chi-squared statistic** and **mutial information (MI)**, whcih is based on the ideas of **entropy**.<br><br>**IMPORTANT!** You need to be done with the previous EXERCISE to be able to continue. <br><br>We need to import the feature selection function **SelectKBest** first. This function selects k best features based on the results of a test (in our case, we will use chi-squared and mutual information). Also, we need the **chi2** function and **mutual_info_classif** functions:

In [18]:
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


First, we need to specify that we will use SelectKBest function with the chi-squared statistic for feature selection (by setting parameter "score_func") and indicate the number of best features we want to find (by setting parameter "k"). We'll select 10,000 best features, so **k = 10,000**:<br><br>
*Look up the documentation for SelectKBest if needed: https://bit.ly/2Re54Ch*

In [20]:
chi2_kbest = SelectKBest(score_func = chi2, k = 10000)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Now, let's find the best features. To do that, we call the **fit_transform** method with our defined kbest function. The fit_transform method takes **2 inputs**: the vectorized representation of the data (NORM_corpus_train_bow) and the array of class labels contained in the object corpus_train.target. To see the chi-squared scores for the features use method **scores_**:

In [21]:
NORM_corpus_train_bow_chi2_BEST = chi2_kbest.fit_transform(NORM_corpus_train_bow, corpus_train.target)
chi2_kbest.scores_

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([2.43001686, 2.48287671, 4.96575342, ..., 2.43001686, 4.86003373,
       4.96575342])

So, which features are best? The **get_support** method with parameter *indices* set to True will return the indecies of the best k features:

In [22]:
chi2_best_features_ind = chi2_kbest.get_support(indices=True)
chi2_best_features_ind

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([    2,     5,     7, ..., 21052, 21059, 21060])

What are the names of the best features, according to the chi-squared statistics?

In [24]:
chi2_best_features_names = np.array(bow_vectorizer.get_feature_names())[chi2_best_features_ind]

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's have a look at the data vectorized using best features selected based on the chi-squared statistic:

In [25]:
#convert vectorized data to a dataframe and give columns their names
X_train_bow_chi2_BEST_table = pd.DataFrame(data = NORM_corpus_train_bow_chi2_BEST.todense(), columns = chi2_best_features_names)
X_train_bow_chi2_BEST_table.head(5)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,000usd,00pm,023b,0x,1024x768,10bps,10km,10m,110m,115m,...,zoroaster,zoroastrian,zoroastrianism,zoroastrians,zubin,zuck,zullen,zurvanism,zwarte,zyxel
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### **<font color=green>EXERCISE 3:</font>**
**<font color=green>3.1. Select best features using the mutual information statistic. Follow the steps we used for the chi-squared statistic. The first line, showing how to specify the SelectKBest function using mutual information, is provided. You need to complete the script.</font>** <br><br>
*Note: you can check out the documentation for mutual information function for categorical data here: https://bit.ly/2JGgeeT* <br><br>
Your code (add more lines):


In [26]:
MI_kbest = SelectKBest(score_func = mutual_info_classif, k = 10000)

NORM_corpus_train_mutual_info_BEST = MI_kbest.fit_transform(NORM_corpus_train_bow, corpus_train.target)
#MI_kbest.scores_
mutual_info_best_features_ind = MI_kbest.get_support(indices=True)
mutual_info_best_features_names = np.array(bow_vectorizer.get_feature_names())[mutual_info_best_features_ind]
#sklearn.feature_selection.mutual_info_classif(X, y, *, discrete_features='auto', n_neighbors=3, copy=True, random_state=None)[source]
X_train_mutual_info_BEST_table = pd.DataFrame(data = NORM_corpus_train_mutual_info_BEST.todense(), columns = mutual_info_best_features_names)
X_train_mutual_info_BEST_table.head(5)

mutual_info_best_features_ind_set = set(mutual_info_best_features_ind)
chi2_best_features_ind_set = set(chi2_best_features_ind)
len(chi2_best_features_ind_set.intersection(mutual_info_best_features_ind_set))/10000


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.8592

**<font color=green>3.2. Do mutual information approach and chi-squared statistic select the same best features?</font>** <br>

Your answer:

85.92% of the best index are the same.

### Feature Extraction for TEST Data

Let's normalize and vectorize the **TEST** text corpus (corpus_test.data) using the Bag-of-Words method. <br><br>First, we normalize the testing corpus and call it NORM_corpus_test:

In [27]:
NORM_corpus_test = normalize_corpus(corpus_test.data)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Now let's vectorize the normalized test corpus, NORM_corpus_test. <br><br>
**IMPORTANT! For transforming test data, you'll use features extracted from the training corpus [You do NOT want to create new feature based on your test data].** <br><br>Therefore: <br> 1) do **not** define a new vectorizer, use the one used on training data <br>2) use method **.transform** (not .fit_trandform) with your vectorizer to vectorize the test data.

In [28]:
NORM_corpus_test_bow = bow_vectorizer.transform(NORM_corpus_test)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


We'll use the best features selected by **chi-squared statistic**. Now we need to pick up from the above Bag-of-Words matrix exactly the same best features we selected for our training dataset:

In [40]:
NORM_corpus_test_bow_chi2_BEST = chi2_kbest.transform(NORM_corpus_test_bow)
type(NORM_corpus_test_bow)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


scipy.sparse.csr.csr_matrix

### Text Classification

We'll train a text classification model that categorizes documents into 4 classes: religion, atheism, science and computer graphics. We will use a Naive Bayes Classifier and Support Vector Machines (SVM), the last one is optional.

#### Naive Bayes Classifier

Make the **MultinomialNB** packages available:

In [30]:
from sklearn.naive_bayes import MultinomialNB

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Define the Naive Bayes classifier by specifiying the hyperparameter alpha and call the classifier NB_tc:<br><br>
*Note: you can set the hyperparameter alpha to an optimal value by trying different values > 0. With alpha = 0, you model will assign a probability of zero to a document in the test data if the document contains a feature not found in the training data.*

In [31]:
NB_tc = MultinomialNB(alpha=0.1) 

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Let's train the model using the best features selected using the **chi-squared statistics**:

In [32]:
NB_tc.fit(NORM_corpus_train_bow_chi2_BEST, corpus_train.target)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


MultinomialNB(alpha=0.1)

In [33]:
predicted_nb_chi2_best = NB_tc.predict(NORM_corpus_test_bow_chi2_BEST)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Evaluate the predictive power for the Naive Bayes classifier using chi-squared k=10,000 best features:

In [34]:
cm_chi2_best = metrics.confusion_matrix(corpus_test.target, predicted_nb_chi2_best)
print("Confusion matrix: \n", pd.DataFrame(data = cm_chi2_best , 
                                           columns = corpus_train.target_names,
                                           index = corpus_train.target_names),"\n")
print("Accuracy rate: ", metrics.accuracy_score(corpus_test.target, predicted_nb_chi2_best),"\n") 

Confusion matrix: 
                     alt.atheism  comp.graphics  sci.space  talk.religion.misc
alt.atheism                 231              7         25                  56
comp.graphics                15            349         23                   2
sci.space                    20             18        349                   7
talk.religion.misc           77              5         19                 150 

Accuracy rate:  0.7974870657797487 



/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### **<font color=green>EXERCISE 4:</font>**
**<font color=green>4.1. Train the Naive Bayes classifier without doing feature selection, that is use all the features available in the normalized corpus. </font>** <br><br>
Your code:

In [44]:
NB_tc.fit(NORM_corpus_train_bow, corpus_train.target)

predicted_nb_no_sele = NB_tc.predict(NORM_corpus_test_bow)


/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
cm_no_sele = metrics.confusion_matrix(corpus_test.target, predicted_nb_no_sele)
print("Confusion matrix: \n", pd.DataFrame(data = cm_no_sele , 
                                           columns = corpus_train.target_names,
                                           index = corpus_train.target_names),"\n")
print("Accuracy rate: ", metrics.accuracy_score(corpus_test.target, predicted_nb_no_sele),"\n") 

Confusion matrix: 
                     alt.atheism  comp.graphics  sci.space  talk.religion.misc
alt.atheism                 227              6         24                  62
comp.graphics                12            350         23                   4
sci.space                    19             18        348                   9
talk.religion.misc           79              8         18                 146 

Accuracy rate:  0.7915742793791575 



/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**<font color=green>What accuracy do you get? If a classifier did a mistake and misclassified a "Computer Graphics" post, to which class such a post was mistakenly assigned, typically? What about a post on the "Atheism" topic? </font>** <br><br>

In [ ]:
The accuracy is 79.16%.
If a classifier did a mistake and misclassified a "Computer Graphics" post, most likely, it will be assigned to the
class sci.sapce, and then alt.atheism, and at last, talk.religion.misc.
For an alt.atheism topic, it will, most likely, be misclassified to talk.religion.misc.

**<font color=green>4.2. (OPTIONAL) Train the Naive Bayes classifier feature selection based on mutual information (MI). What accuracy do you get?**<br><br>
Your code:

In [48]:
NORM_corpus_test_bow_mi_BEST = MI_kbest.transform(NORM_corpus_test_bow)

NB_tc.fit(NORM_corpus_train_mutual_info_BEST, corpus_train.target)

predicted_nb_mi = NB_tc.predict(NORM_corpus_test_bow_mi_BEST)

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Your answer:

In [49]:
cm_no_mi = metrics.confusion_matrix(corpus_test.target, predicted_nb_mi)
print("Confusion matrix: \n", pd.DataFrame(data = cm_no_mi , 
                                           columns = corpus_train.target_names,
                                           index = corpus_train.target_names),"\n")
print("Accuracy rate: ", metrics.accuracy_score(corpus_test.target, predicted_nb_mi),"\n") 

Confusion matrix: 
                     alt.atheism  comp.graphics  sci.space  talk.religion.misc
alt.atheism                 237              7         24                  51
comp.graphics                10            351         24                   4
sci.space                    20             18        349                   7
talk.religion.misc           84              7         19                 141 

Accuracy rate:  0.7967479674796748 



/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [51]:
The accuracy is 79.67%, higher than the two methods above.

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### **<font color=green>EXERCISE 5 (OPTIONAL):</font>**
**<font color=green>5.1. Vectorize the data using the TF-IDF approach, with and without feature selection, and train and test the Naive Bayes classifier. What are your results? </font>** <br><br>

Your code:

Your answer: